# MARKETING ANALYSIS | CUSTOMER SEGMENTATION

## Importing Libraries

In [142]:
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt
%matplotlib inline
import seaborn as sns
import warnings
warnings.filterwarnings('ignore')
pd.set_option('display.max_columns', None)
import plotly.graph_objs as go
from plotly.subplots import make_subplots

# 

# CUSTOMER SEGMENTATION OF ONLINE RETAIL WITH RFM

### Data Source: https://archive.ics.uci.edu/dataset/352/online+retail

#

## Data Importing and Inspecting 

In [91]:
retail = pd.read_excel('online_retail.xlsx')
retail.head()

,InvoiceNo,StockCode,Description,Quantity,InvoiceDate,UnitPrice,CustomerID,Country
0,536365,85123A,WHITE HANGING HEART T-LIGHT HOLDER,6,2010-12-01 08:26:00,2.55,17850.0,United Kingdom
1,536365,71053,WHITE METAL LANTERN,6,2010-12-01 08:26:00,3.39,17850.0,United Kingdom
2,536365,84406B,CREAM CUPID HEARTS COAT HANGER,8,2010-12-01 08:26:00,2.75,17850.0,United Kingdom
3,536365,84029G,KNITTED UNION FLAG HOT WATER BOTTLE,6,2010-12-01 08:26:00,3.39,17850.0,United Kingdom
4,536365,84029E,RED WOOLLY HOTTIE WHITE HEART.,6,2010-12-01 08:26:00,3.39,17850.0,United Kingdom


In [92]:
# My working on Dataframe

retail_df = retail.copy()

## Data Preperation & Exploration

In [119]:
# Defining missing values analysis function

def missing_values_analysis(df):
    na_columns = [col for col in df.columns if df[col].isnull().sum() > 0]
    n_miss = df[na_columns].isnull().sum().sort_values(ascending = True)
    ratio_ = (df[na_columns].isnull().sum() / df.shape[0]*100).sort_values(ascending = True)
    missing_df = pd.concat([n_miss , np.round(ratio_,2)], axis = 1, keys=['Missing Values','Ratio'])
    return missing_df

# Defining Overview function

def dataframe_overview(df, head=5, tail=5):
    print('SHAPE'.center(82,'~'))
    print('Row: {}'.format(df.shape[0]))
    print('Column: {}'.format(df.shape[1]))
    print('TYPES'.center(82,'~'))
    print(df.dtypes)
    print('MISSING VALUES'.center(82,'~'))
    print(missing_values_analysis(df))
    print('DUPLICATION'.center(83,'~'))
    print(df.duplicated().sum())
    print('QUANTILES'.center(82,'~'))
    print(df.quantile([0,0.05, 0.5, 0.95, 0.99, 1]).T)

## Categorical Data Overview Setup

In [149]:
# My Color Palette

colors = ['#FF5733', '#33FF57', '#5733FF', '#FF33A1', '#33A1FF', '#FFD700', '#8A2BE2', '#00CED1', '#FF6347', '#4CAF50',
          '#FF4500', '#1E90FF', '#FF1493', '#8B008B', '#20B2AA', '#7CFC00', '#9ACD32', '#FF8C00', '#48D1CC', '#9370DB',
          '#556B2F', '#9400D3', '#00FF7F', '#DC143C', '#B0E0E6', '#DAA520']

# Function Defination

def categorical_variable_summary(df, column_name):
    fig = make_subplots(rows = 1,
                       cols = 2,
                       subplot_titles = ('Countplot','Percentages'),
                       specs = [[{'type':'xy'},{'type':'domain'}]])
    
# Bar Plot Defination 

    fig.add_trace(go.Bar(x = [str(i) for i in df[column_name].value_counts().index],
                         y = df[column_name].value_counts().values.tolist(),
                         text = df[column_name].value_counts().values.tolist(),
                         textfont = dict(size = 15),
                         textposition = 'auto',
                         showlegend = False,
                         marker = dict(color = colors,
                                     line = dict(color = '#DBE6EC',
                                                width = 1))),
                row = 1, col = 1)
    
    
# Pie Plot Defination

    fig.add_trace(go.Pie(labels = [str(i) for i in df[column_name].value_counts().keys()],
                         values = df[column_name].value_counts().values,
                         textfont = dict(size = 20),
                         textposition = 'auto',
                         showlegend = False,
                         marker = dict(colors = colors)),
                 row = 1, col = 2)
    
    fig.update_layout(title={'text':column_name,
                             'y' : 0.9,
                             'x' : 0.5,
                             'xanchor':'center',
                             'yanchor':'top'},
                     template = 'plotly_white')
    fig.show()

## Retail Data Overview

In [120]:
# retail data overview

dataframe_overview(retail_df)

~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~SHAPE~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~
Row: 397924
Column: 9
~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~TYPES~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~
InvoiceNo              object
StockCode              object
Description            object
Quantity                int64
InvoiceDate    datetime64[ns]
UnitPrice             float64
CustomerID            float64
Country                object
Sales                 float64
dtype: object
~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~MISSING VALUES~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~
Empty DataFrame
Columns: [Missing Values, Ratio]
Index: []
~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~DUPLICATION~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~
5192
~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~QUANTILES~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~
               0.00      0.05      0.50     0.95      0.99       1.00
Quantity        1.0      1.00      6.00     36.0    120.00   80995.00
UnitPrice       0.0      0.42      1.95      8.5     14.95    8142.75
Cust

In [95]:
# Drop missing Customer IDs

retail_df.dropna(inplace=True)

In [96]:
# Exploring The Products

# Approach 1
retail_df.groupby('Description')['Quantity'].sum().reset_index().sort_values('Quantity',ascending = False)

# Approach 2
retail_df.groupby('Description').agg({'Quantity':'sum'}).sort_values('Quantity',ascending = False)

,Quantity
Description,
WORLD WAR 2 GLIDERS ASSTD DESIGNS,53215
JUMBO BAG RED RETROSPOT,45066
ASSORTED COLOUR BIRD ORNAMENT,35314
WHITE HANGING HEART T-LIGHT HOLDER,34147
PACK OF 72 RETROSPOT CAKE CASES,33409
...,...
PINK POODLE HANGING DECORATION,-12
CRUK Commission,-16
ASSORTED TUTTI FRUTTI ROUND BOX,-24


In [97]:
# Check on the canceled orders

retail_df['InvoiceNo'].str.contains('C').count()

8905

In [98]:
# Drop canceled orders

retail_df = retail_df[~retail_df['InvoiceNo'].str.contains('C', na = False)]
retail_df.head()

,InvoiceNo,StockCode,Description,Quantity,InvoiceDate,UnitPrice,CustomerID,Country
0,536365,85123A,WHITE HANGING HEART T-LIGHT HOLDER,6,2010-12-01 08:26:00,2.55,17850.0,United Kingdom
1,536365,71053,WHITE METAL LANTERN,6,2010-12-01 08:26:00,3.39,17850.0,United Kingdom
2,536365,84406B,CREAM CUPID HEARTS COAT HANGER,8,2010-12-01 08:26:00,2.75,17850.0,United Kingdom
3,536365,84029G,KNITTED UNION FLAG HOT WATER BOTTLE,6,2010-12-01 08:26:00,3.39,17850.0,United Kingdom
4,536365,84029E,RED WOOLLY HOTTIE WHITE HEART.,6,2010-12-01 08:26:00,3.39,17850.0,United Kingdom


In [99]:
retail_df["Sales"] = retail_df["Quantity"]*retail_df['UnitPrice']
retail_df.head()

,InvoiceNo,StockCode,Description,Quantity,InvoiceDate,UnitPrice,CustomerID,Country,Sales
0,536365,85123A,WHITE HANGING HEART T-LIGHT HOLDER,6,2010-12-01 08:26:00,2.55,17850.0,United Kingdom,15.30
1,536365,71053,WHITE METAL LANTERN,6,2010-12-01 08:26:00,3.39,17850.0,United Kingdom,20.34
2,536365,84406B,CREAM CUPID HEARTS COAT HANGER,8,2010-12-01 08:26:00,2.75,17850.0,United Kingdom,22.00
3,536365,84029G,KNITTED UNION FLAG HOT WATER BOTTLE,6,2010-12-01 08:26:00,3.39,17850.0,United Kingdom,20.34
4,536365,84029E,RED WOOLLY HOTTIE WHITE HEART.,6,2010-12-01 08:26:00,3.39,17850.0,United Kingdom,20.34


# RFM Analysis

In [103]:
# import datetime library

import datetime as dt

retail_df['InvoiceDate'].max()

Timestamp('2011-12-09 12:50:00')

In [105]:
# Create a mock today_date

today_date = dt.datetime(2011,12,14)

In [110]:
# Calculate the RFM Model

rfm = retail_df.groupby('CustomerID').agg({'InvoiceDate': lambda date: (today_date - date.max()).days,
                                           'InvoiceNo': lambda num: num.nunique(),
                                           'Sales': lambda sales: sales.sum()})
rfm

,InvoiceDate,InvoiceNo,Sales
CustomerID,,,
12346.0,329,1,77183.60
12347.0,6,7,4310.00
12348.0,79,4,1797.24
12349.0,22,1,1757.55
12350.0,314,1,334.40
...,...,...,...
18280.0,281,1,180.60
18281.0,184,1,80.82
18282.0,11,2,178.05


In [111]:
# Change Column Names

rfm.columns = ['Recency','Frequency','Monetary']
rfm.head()

,Recency,Frequency,Monetary
CustomerID,,,
12346.0,329,1,77183.60
12347.0,6,7,4310.00
12348.0,79,4,1797.24
12349.0,22,1,1757.55
12350.0,314,1,334.40


In [112]:
# Include customers only with Monetary Values

rfm = rfm[rfm['Monetary']>0]
rfm.head()

,Recency,Frequency,Monetary
CustomerID,,,
12346.0,329,1,77183.60
12347.0,6,7,4310.00
12348.0,79,4,1797.24
12349.0,22,1,1757.55
12350.0,314,1,334.40


In [114]:
# RFM Description
rfm.describe().T

,count,mean,std,min,25%,50%,75%,max
Recency,4338.0,96.059474,100.012264,4.00,21.000,54.000,145.75,377.00
Frequency,4338.0,4.272706,7.706221,1.00,1.000,2.000,5.00,210.00
Monetary,4338.0,2054.266460,8989.230441,3.75,307.415,674.485,1661.74,280206.02


In [125]:
# RFM Scores

rfm['Recency_Score'] = pd.qcut(rfm['Recency'], 5, labels = [5,4,3,2,1])
rfm['Frequency_Score'] = pd.qcut(rfm['Frequency'].rank(method = 'first'), 5, labels = [1,2,3,4,5])
rfm['Monetary_Score'] = pd.qcut(rfm['Monetary'], 5, labels = [1,2,3,4,5])
rfm.head()

,Recency,Frequency,Monetary,Recency_Score,Frequency_Score,Monetary_Score
CustomerID,,,,,,
12346.0,329,1,77183.60,1,1,5
12347.0,6,7,4310.00,5,5,5
12348.0,79,4,1797.24,2,4,4
12349.0,22,1,1757.55,4,1,4
12350.0,314,1,334.40,1,1,2


In [126]:
# RFM SCORE

rfm['RFM_SCORE'] = (rfm['Recency_Score'].astype(str) + rfm['Frequency_Score'].astype(str) )
rfm.head(10)

,Recency,Frequency,Monetary,Recency_Score,Frequency_Score,Monetary_Score,RFM_SCORE
CustomerID,,,,,,,
12346.0,329,1,77183.60,1,1,5,11
12347.0,6,7,4310.00,5,5,5,55
12348.0,79,4,1797.24,2,4,4,24
12349.0,22,1,1757.55,4,1,4,41
12350.0,314,1,334.40,1,1,2,11
12352.0,40,8,2506.04,3,5,5,35
12353.0,208,1,89.00,1,1,1,11
12354.0,236,1,1079.40,1,1,4,11
12355.0,218,1,459.40,1,1,2,11


In [150]:
categorical_variable_summary(rfm, 'RFM_SCORE')

## Segmenting Customers

In [134]:
#Case-When-Then Statement Corresponding 

seg_map = {
    r'[1-2][1-2]': 'hibernating_potatoes',
    r'[1-2][3-4]': 'living_on_the_edge',
    r'[1-2]5': 'living_in_a_fantasy',
    r'3[1-2]': 'about_to_Zzz',
    r'33': 'almost_dreaming',
    r'[3-4][4-5]': 'super_duper_fans',
    r'41': 'showing_promise_like_a_boss',
    r'51': 'brand_new_buddies',
    r'[4-5][2-3]': 'future_legends_in_training',
    r'5[4-5]': 'ultimate_champions',
}
rfm['segment'] = rfm.RFM_SCORE.replace(seg_map, regex=True)
rfm.head(10)

,Recency,Frequency,Monetary,Recency_Score,Frequency_Score,Monetary_Score,RFM_SCORE,segment
CustomerID,,,,,,,,
12346.0,329,1,77183.60,1,1,5,11,hibernating_potatoes
12347.0,6,7,4310.00,5,5,5,55,ultimate_champions
12348.0,79,4,1797.24,2,4,4,24,living_on_the_edge
12349.0,22,1,1757.55,4,1,4,41,showing_promise_like_a_boss
12350.0,314,1,334.40,1,1,2,11,hibernating_potatoes
12352.0,40,8,2506.04,3,5,5,35,super_duper_fans
12353.0,208,1,89.00,1,1,1,11,hibernating_potatoes
12354.0,236,1,1079.40,1,1,4,11,hibernating_potatoes
12355.0,218,1,459.40,1,1,2,11,hibernating_potatoes


In [151]:
categorical_variable_summary(rfm, 'segment')

## Segmentations Analysis

In [138]:
rfm[['segment','Recency','Frequency','Monetary']].groupby('segment').agg(['min','count','max'])

Recency            Frequency            Monetary  \
                                min count  max       min count  max      min   
segment                                                                        
about_to_Zzz                     37   352   75         1   352    2     6.20   
almost_dreaming                  37   187   75         2   187    3     6.90   
brand_new_buddies                 4    42   16         1    42    1    89.94   
future_legends_in_training        4   484   36         1   484    3    20.80   
hibernating_potatoes             76  1071  377         1  1071    2     3.75   
living_in_a_fantasy              76    63  376         6    63   34    70.02   
living_on_the_edge               76   593  377         2   593    6    52.00   
showing_promise_like_a_boss      18    94   36         1    94    1    30.00   
super_duper_fans                 18   819   75         3   819   63    36.56   
ultimate_champions                4   633   16         3   633  210   201.12   

                                              
                            count        max  
segment                                       
about_to_Zzz                  352    6207.67  
almost_dreaming               187   12601.83  
brand_new_buddies              42    3861.00  
future_legends_in_training    484  168472.50  
hibernating_potatoes         1071   77183.60  
living_in_a_fantasy            63   10254.18  
living_on_the_edge            593   44534.30  
showing_promise_like_a_boss    94    1757.55  
super_duper_fans              819  124914.53  
ultimate_champions            633  280206.02